In [49]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [50]:
import scipy.stats as st
from functions import *
from graphics import *
from statistics_tests import *
import pandas as pd

df_final_demo = get_final_demo_df()

df_web_data_concat = get_web_data_df()

df_final_exp = get_final_exp_df()

df_all = get_df_all()

LIMPIEZA DE DATOS

In [51]:
df_all = df_all.drop("clnt_tenure_yr", axis=1)      # We don´t need this column because is derivative from clnt_tenure_mnth

QUITAMOS OUTLIERS DEL DF_ALL Y DEL QUE NO TIENE DUPLICADOS

In [52]:
df_all = outlier_slayer(df_all)

In [53]:
# df_all["time_diff"] = df_all.groupby(["client_id", "visit_id"])["date_time"].shift(1) - df_all["date_time"]

In [54]:
df_all["time_diff"] = abs(df_all.groupby(["client_id", "visit_id"])["date_time"].diff().shift(-1).fillna(pd.Timedelta(seconds=0)) ).dt.total_seconds()

In [55]:
df_inter = df_all.copy()
df_inter.drop_duplicates(subset=["visit_id", "process_step"], keep="last", inplace=True)

In [56]:
df_all["time_diff_cleaned"] = abs(df_inter.groupby(["client_id", "visit_id"])["date_time"].diff().shift(-1).fillna(pd.Timedelta(seconds=0)) ).dt.total_seconds()

In [57]:
df_all["repetition"] = df_all.duplicated(subset=["visit_id", "process_step"], keep="last").astype(int)          #0 = no, 1 = yes

In [58]:
df_all["last_step"] = df_all.groupby("visit_id")["process_step"].transform("max")

In [59]:
df_all.to_csv("data/pruebaNoche.csv")

In [63]:
df_all.iloc[20:40]

,client_id,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth,visitor_id,visit_id,process_step,date_time,variation,time_diff,time_diff_cleaned,repetition,last_step
20,1439522,64.0,32.0,U,2.0,52467.79,6.0,9.0,245908175_84333642537,898099666_87299892721_263146,0,2017-04-06 09:28:02,1,0.0,0.0,0,3
21,1439522,64.0,32.0,U,2.0,52467.79,6.0,9.0,407067363_48226131468,968288276_7230824336_666089,0,2017-04-07 09:16:00,1,0.0,0.0,0,0
22,1562045,198.0,49.0,M,2.0,67454.65,3.0,6.0,391202328_71710993352,71954691_44810191073_495385,0,2017-04-06 19:03:46,1,0.0,0.0,0,0
23,5126305,145.0,33.0,F,2.0,103671.75,0.0,3.0,432077852_35036160885,942996157_39527225999_589383,0,2017-06-14 14:22:31,0,0.0,0.0,0,0
24,3727881,71.0,30.5,U,2.0,23915.60,0.0,3.0,875448077_54387937735,717519432_65547872352_613051,4,2017-03-29 14:00:45,0,63.0,131.0,0,4
25,3727881,71.0,30.5,U,2.0,23915.60,0.0,3.0,875448077_54387937735,717519432_65547872352_613051,3,2017-03-29 13:59:42,0,46.0,NaN,1,4
26,3727881,71.0,30.5,U,2.0,23915.60,0.0,3.0,875448077_54387937735,717519432_65547872352_613051,2,2017-03-29 13:58:56,0,22.0,NaN,1,4
27,3727881,71.0,30.5,U,2.0,23915.60,0.0,3.0,875448077_54387937735,717519432_65547872352_613051,3,2017-03-29 13:58:34,0,219.0,219.0,0,4
28,3727881,71.0,30.5,U,2.0,23915.60,0.0,3.0,875448077_54387937735,717519432_65547872352_613051,2,2017-03-29 13:54:55,0,10.0,10.0,0,4
29,3727881,71.0,30.5,U,2.0,23915.60,0.0,3.0,875448077_54387937735,717519432_65547872352_613051,1,2017-03-29 13:54:45,0,30.0,30.0,0,4


In [61]:
# count = df_all[(df_all["process_step"] == 4) & 
#                 (df_all["repetition"] == 0) & 
#                 (df_all["variation"] == 1)].shape[0]

# count

In [62]:
# df_inter["time_diff_cleaned"] = abs(df_inter.groupby(["client_id", "visit_id"])["date_time"].diff().shift(-1).fillna(pd.Timedelta(seconds=0)) ).dt.total_seconds()